In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-8sz1f7zg7oy0ui"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


#### Temasek Foundation Dataset

In [2]:
# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.load_data import prep_tf_node 

tf_meta_prompt, test_cases, tf_metric_map = prep_tf_node(prompt_mode=True)

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 
node = EvolNode(tf_meta_prompt, None, None, get_response=get_endpoint_response, test_cases=test_cases[:90],
                custom_metric_map=tf_metric_map) # setting manual test cases


In [3]:
# node.evolve("i1", batch_size=5)

import time 
query_node = True 
replace = True 
feedback = ""
batch_size = 5
self = node 
method = "i1"
parents = None 

start_time = time.time()
# Query once
offsprings = [] 
if query_node:
    self.query_nodes(ignore_self=replace, self_func_name=self.meta_prompt.func_name)
query_end_time = time.time()
query_time = query_end_time - start_time
print(f"     :: Query time: {query_time:.2f}s")

# Evolve many times
reasonings, codes = self._evolve(method, parents, feedback=feedback, batch_size=batch_size)
evolve_end_time = time.time()
evolve_time = evolve_end_time - query_end_time
print(f"     :: Evolution time: {evolve_time:.2f}s")

self.reasonings = reasonings
self.codes = codes

     :: Query time: 3.48s


Processing LLM queries: 100%|██████████| 5/5 [00:08<00:00,  1.71s/it]

 :: Total time elapsed: 8.54s, 0 errors
     :: Evolution time: 8.57s


In [4]:
max_tries, num_runs = 3, 2
test_cases = node.test_cases
test_inputs = [case[0] for case in test_cases]
input_dicts = test_inputs 

In [38]:
code = 'def generate_prompt(project_description):\n    prompt = (\n        f"Evaluate the grant application described as {project_description}. Provide a decision (Yes, No, Maybe) and a brief comment explaining the decision. Assume the AI only has access to the prompt and the tools provided. Please output a json-style response in markdown format like this: {{\'label\': \'str(...)\',\'comment\': \'str(...\')}}. Do not include any additional reasoning or output. Use the output from search_google as a guide for your response."\n        )\n    return prompt'
burn_code = 'def generate_prompt(project_description):\n    import time\n    time.sleep(10)\n    prompt = (\n        f"Evaluate the grant application described as {project_description}. Provide a decision (Yes, No, Maybe) and a brief comment explaining the decision. Assume the AI only has access to the prompt and the tools provided. Please output a json-style response in markdown format like this: {{\'label\': \'str(...)\',\'comment\': \'str(...\')}}. Do not include any additional reasoning or output. Use the output from search_google as a guide for your response."\n        )\n    return prompt'
codes = [burn_code] * 5 
self.codes = codes

In [46]:
import types 
from methods.meta_execute import Timeout_
from collections import defaultdict
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
from typing import Optional


def process_single_input(code, input_tuple, max_tries):
    input_index, input_dict = input_tuple
    prompts = []
    errors = []
    
    mod = types.ModuleType('dynamic_module')
    try:
        # Remove the Timeout_ wrapper
        exec(code, mod.__dict__)
        func_name = "generate_prompt"
        assert func_name in mod.__dict__, f"Function {func_name} not found in code"
        prompt_func = mod.__dict__[func_name]
      
        prompt = prompt_func(**input_dict)               
        for _ in range(max_tries):
            prompts.append(prompt)
        errors = []
    except Exception as e:
        errors.append(str(e))
        prompts = []
        
    return input_index, prompts, errors

def process_single_input_with_timeout(code, input_tuple, max_tries, timeout: int = 3):
    input_index, input_dict = input_tuple
    prompts = []
    errors = []
    
    mod = types.ModuleType('dynamic_module')
    try:
        # Remove the Timeout_ wrapper
        exec(code, mod.__dict__)
        func_name = "generate_prompt"
        assert func_name in mod.__dict__, f"Function {func_name} not found in code"
        prompt_func = mod.__dict__[func_name]
        
        with Timeout_(seconds=timeout):
            prompt = prompt_func(**input_dict)               
        
        for _ in range(max_tries):
            prompts.append(prompt)
        errors = []
    except Exception as e:
        errors.append(str(e))
        prompts = []
        
    return input_index, prompts, errors


def process_all_inputs(codes, input_dicts, max_tries, timeout: int = 3):
    outputs_per_code_per_test = defaultdict(lambda: defaultdict(list))
    errors_per_code_per_test = defaultdict(lambda: defaultdict(list))
    
    total_iterations = len(codes) * len(input_dicts)
    tasks = [
        (code_index, code, input_index, input_dict) 
        for code_index, code in enumerate(codes)
        for input_index, input_dict in enumerate(input_dicts)
    ]
    
    with ThreadPoolExecutor(max_workers=min(32, total_iterations)) as executor:
        futures = {
            executor.submit(
                process_single_input, 
                code, 
                (input_index, input_dict), 
                max_tries
            ): (code_index, input_index)
            for code_index, code, input_index, input_dict in tasks
        }
        
        with tqdm(total=total_iterations, desc="Processing LLM queries") as pbar:
            for future in as_completed(futures):
                code_index, input_index = futures[future]
                try:
                    # Add timeout here
                    input_index, prompts, errors = future.result(timeout=timeout)
                    outputs_per_code_per_test[code_index][input_index].extend(prompts)
                    errors_per_code_per_test[code_index][input_index].extend(errors)
                except concurrent.futures.TimeoutError:
                    errors_per_code_per_test[code_index][input_index].append(f"Execution timed out after {timeout} seconds")
                except Exception as e:
                    errors_per_code_per_test[code_index][input_index].append(str(e))
                pbar.update(1)
    
    return outputs_per_code_per_test, errors_per_code_per_test

In [47]:
# process single input 
# input_index, prompts, errors = process_single_input(codes[0], (0, input_dicts[0]), max_tries)


# Help me benchmark the difference in below two methods (double loop vs concurrent implementation)

# Double Loop Implementation
outputs_per_code_per_test_loop = defaultdict(lambda: defaultdict(list))
errors_per_code_per_test_loop = defaultdict(lambda: defaultdict(list))

import time
start_time = time.time()

# total_iterations = len(codes) * len(input_dicts)
# with tqdm(total=total_iterations, desc="Processing LLM queries") as pbar:
#     for code_index, code in enumerate(codes):
#         for input_index, input_dict in enumerate(input_dicts):
#             input_index, prompts, errors = process_single_input(code, (input_index, input_dict), max_tries)
#             outputs_per_code_per_test_loop[code_index][input_index].extend(prompts)
#             errors_per_code_per_test_loop[code_index][input_index].extend(errors)
#             pbar.update(1)

# loop_time = time.time() - start_time
# print(f"\nDouble loop implementation took {loop_time:.2f} seconds")

# Concurrent Implementation
start_time = time.time()
outputs_per_code_per_test_concurrent, errors_per_code_per_test_concurrent = process_all_inputs(codes, input_dicts, max_tries)
concurrent_time = time.time() - start_time

print(f"Concurrent implementation took {concurrent_time:.2f} seconds")
# print(f"Speed improvement: {loop_time/concurrent_time:.2f}x faster with concurrent implementation")

# # Verify results are the same
# assert outputs_per_code_per_test_loop == outputs_per_code_per_test_concurrent, "Output results differ!"
# assert errors_per_code_per_test_loop == errors_per_code_per_test_concurrent, "Error results differ!"

Processing LLM queries: 100%|██████████| 450/450 [02:30<00:00,  3.00it/s]

Concurrent implementation took 150.07 seconds


In [49]:
3 * 450 / 150

9.0